In [1]:
from shapely.geometry import Point
from collections import Counter
import seaborn as sns
import datetime as datetime
from datetime import date
import io
import re
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
mma_data = pd.read_csv('UFC.csv')

/opt/anaconda3/envs/geospatial/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (144,145) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
mma_data

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_index_ape,B_index_ape,R_index_ape2,B_index_ape2
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.0,...,Orthodox,170.18,177.80,135.0,31.0,27.0,1.04,1.03,7.62,5.08
1,Trevin Giles,Roman Dolidze,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.0,...,Orthodox,182.88,187.96,185.0,32.0,28.0,1.03,1.03,5.08,5.08
2,Tai Tuivasa,Harry Hunsucker,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,...,Southpaw,187.96,190.50,264.0,32.0,28.0,1.01,1.01,2.54,2.54
3,Cheyanne Buys,Montserrat Conejo,Mark Smith,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,...,Switch,160.02,160.02,115.0,28.0,25.0,1.0,1.02,0.00,2.54
4,Marion Reneau,Macy Chiasson,Mark Smith,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.0,...,Orthodox,167.64,172.72,135.0,29.0,43.0,1.03,1.01,5.08,2.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,Remco Pardoel,Alberta Cerra Leon,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Southpaw,190.50,NaN,260.0,NaN,NaN,0,0,-190.50,-172.72
6008,Orlando Wiet,Robert Lucarelli,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Southpaw,177.80,NaN,170.0,NaN,NaN,0,0,-177.80,-187.96
6009,Johnny Rhodes,David Levicki,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Orthodox,182.88,NaN,210.0,NaN,NaN,0,0,-182.88,-195.58
6010,Patrick Smith,Ray Wizard,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Orthodox,187.96,NaN,225.0,NaN,30.0,0,#DIV/0!,-187.96,0.00


In [4]:
mma_data.info

<bound method DataFrame.info of           R_fighter           B_fighter        Referee       date  \
0      Adrian Yanez       Gustavo Lopez  Chris Tognoni  3/20/2021   
1      Trevin Giles       Roman Dolidze      Herb Dean  3/20/2021   
2       Tai Tuivasa     Harry Hunsucker      Herb Dean  3/20/2021   
3     Cheyanne Buys   Montserrat Conejo     Mark Smith  3/20/2021   
4     Marion Reneau       Macy Chiasson     Mark Smith  3/20/2021   
...             ...                 ...            ...        ...   
6007  Remco Pardoel  Alberta Cerra Leon  John McCarthy  3/11/1994   
6008   Orlando Wiet    Robert Lucarelli  John McCarthy  3/11/1994   
6009  Johnny Rhodes       David Levicki  John McCarthy  3/11/1994   
6010  Patrick Smith          Ray Wizard  John McCarthy  3/11/1994   
6011   Scott Morris      Sean Daugherty  John McCarthy  3/11/1994   

                    location Winner  title_bout       weight_class  B_avg_KD  \
0     Las Vegas, Nevada, USA    Red       False       Banta

In [5]:
win_loss_draw = mma_data[['R_fighter', 'B_fighter','R_wins', 'R_draw', 'R_losses', 'B_wins', 'B_draw', 'B_losses']]

In [6]:
win_loss_draw

,R_fighter,B_fighter,R_wins,R_draw,R_losses,B_wins,B_draw,B_losses
0,Adrian Yanez,Gustavo Lopez,1,0,0,1,0,1
1,Trevin Giles,Roman Dolidze,4,0,2,2,0,0
2,Tai Tuivasa,Harry Hunsucker,4,0,3,0,0,0
3,Cheyanne Buys,Montserrat Conejo,0,0,0,0,0,0
4,Marion Reneau,Macy Chiasson,5,0,6,4,0,1
...,...,...,...,...,...,...,...,...
6007,Remco Pardoel,Alberta Cerra Leon,0,0,0,0,0,0
6008,Orlando Wiet,Robert Lucarelli,0,0,0,0,0,0
6009,Johnny Rhodes,David Levicki,0,0,0,0,0,0
6010,Patrick Smith,Ray Wizard,0,0,0,0,0,0


In [7]:
win_loss_draw.to_csv ('win_loss_draw.csv', index = False)

In [8]:
mma_data['date'] = pd.to_datetime(mma_data['date'])
ufc_unified_rules = mma_data[~(mma_data['date'] < '2000-01-01')]

In [9]:
ufc_unified_rules

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_index_ape,B_index_ape,R_index_ape2,B_index_ape2
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.0,...,Orthodox,170.18,177.80,135.0,31.0,27.0,1.04,1.03,7.62,5.08
1,Trevin Giles,Roman Dolidze,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.0,...,Orthodox,182.88,187.96,185.0,32.0,28.0,1.03,1.03,5.08,5.08
2,Tai Tuivasa,Harry Hunsucker,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,...,Southpaw,187.96,190.50,264.0,32.0,28.0,1.01,1.01,2.54,2.54
3,Cheyanne Buys,Montserrat Conejo,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,...,Switch,160.02,160.02,115.0,28.0,25.0,1.0,1.02,0.00,2.54
4,Marion Reneau,Macy Chiasson,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.0,...,Orthodox,167.64,172.72,135.0,29.0,43.0,1.03,1.01,5.08,2.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5804,Scott Adams,Ian Freeman,Dan Severn,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Heavyweight,NaN,NaN,...,Southpaw,182.88,NaN,225.0,33.0,NaN,0,0,-182.88,-180.34
5805,Jens Pulver,David Velasquez,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Lightweight,NaN,NaN,...,Southpaw,170.18,177.80,145.0,NaN,25.0,1.04,0,7.62,-165.10
5806,Tedd Williams,Steve Judson,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Heavyweight,0.000,1.0,...,Orthodox,185.42,NaN,275.0,NaN,30.0,0,0,-185.42,-180.34
5807,Bob Cook,Tiki Ghosn,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Lightweight,NaN,NaN,...,Orthodox,175.26,NaN,170.0,23.0,NaN,0,0,-175.26,-177.80


In [10]:
ufc_unified_rules.info

<bound method DataFrame.info of           R_fighter          B_fighter        Referee       date  \
0      Adrian Yanez      Gustavo Lopez  Chris Tognoni 2021-03-20   
1      Trevin Giles      Roman Dolidze      Herb Dean 2021-03-20   
2       Tai Tuivasa    Harry Hunsucker      Herb Dean 2021-03-20   
3     Cheyanne Buys  Montserrat Conejo     Mark Smith 2021-03-20   
4     Marion Reneau      Macy Chiasson     Mark Smith 2021-03-20   
...             ...                ...            ...        ...   
5804    Scott Adams        Ian Freeman     Dan Severn 2000-03-10   
5805    Jens Pulver    David Velasquez  John McCarthy 2000-03-10   
5806  Tedd Williams       Steve Judson  John McCarthy 2000-03-10   
5807       Bob Cook         Tiki Ghosn  John McCarthy 2000-03-10   
5808   Lance Gibson     Jermaine Andre  John McCarthy 2000-03-10   

                          location Winner  title_bout       weight_class  \
0           Las Vegas, Nevada, USA    Red       False       Bantamweight   

find earliest date each r fighter appears in the dataset 

In [11]:
r_first_appearance = ufc_unified_rules.groupby('R_fighter')['date'].min().reset_index()

In [12]:
r_first_appearance

,R_fighter,date
0,Aalon Cruz,2020-02-29
1,Aaron Phillips,2014-08-23
2,Aaron Riley,2008-11-15
3,Aaron Rosa,2011-11-12
4,Aaron Simpson,2009-04-01
...,...,...
1447,Zelim Imadaev,2019-11-09
1448,Zhang Lipeng,2014-08-23
1449,Zhang Tiequan,2011-02-26
1450,Zhang Weili,2018-11-24


In [13]:
r_last_appearance = ufc_unified_rules.groupby('R_fighter')['date'].max().reset_index()

In [14]:
r_last_appearance

,R_fighter,date
0,Aalon Cruz,2020-02-29
1,Aaron Phillips,2014-08-23
2,Aaron Riley,2013-07-27
3,Aaron Rosa,2011-11-12
4,Aaron Simpson,2012-10-05
...,...,...
1447,Zelim Imadaev,2019-11-09
1448,Zhang Lipeng,2015-05-16
1449,Zhang Tiequan,2012-11-10
1450,Zhang Weili,2020-03-07


In [15]:
r_time_active = r_last_appearance['date'] - r_first_appearance['date']

In [16]:
r_time_active

0         0 days
1         0 days
2      1715 days
3         0 days
4      1283 days
          ...   
1447      0 days
1448    266 days
1449    623 days
1450    469 days
1451   1799 days
Name: date, Length: 1452, dtype: timedelta64[ns]

df['Age'] = (datetime.datetime.now() - pd.to_datetime(df[['Day','Month','Year']]))

r_time_active[0,0] = (datetime.datetime.now() - pd.to_datetime(r_time_active[[0,0]]))

In [17]:
b_first_appearance = ufc_unified_rules.groupby('B_fighter')['date'].min().reset_index()

In [18]:
b_first_appearance

,B_fighter,date
0,Aalon Cruz,2021-03-06
1,Aaron Brink,2000-11-17
2,Aaron Phillips,2014-05-24
3,Aaron Riley,2002-05-10
4,Aaron Rosa,2011-06-11
...,...,...
1859,Zelim Imadaev,2019-04-13
1860,Zhalgas Zhumagulov,2020-07-11
1861,Zhang Lipeng,2014-03-01
1862,Zhang Weili,2018-08-04


In [19]:
b_last_appearance = ufc_unified_rules.groupby('B_fighter')['date'].max().reset_index()

In [20]:
b_last_appearance

,B_fighter,date
0,Aalon Cruz,2021-03-06
1,Aaron Brink,2000-11-17
2,Aaron Phillips,2020-07-15
3,Aaron Riley,2011-09-24
4,Aaron Rosa,2012-03-02
...,...,...
1859,Zelim Imadaev,2020-09-05
1860,Zhalgas Zhumagulov,2021-01-23
1861,Zhang Lipeng,2014-03-01
1862,Zhang Weili,2019-08-31


In [21]:
b_time_active = b_last_appearance['date'] - b_first_appearance['date']

In [22]:
b_time_active

0         0 days
1         0 days
2      2244 days
3      3424 days
4       265 days
          ...   
1859    511 days
1860    196 days
1861      0 days
1862    392 days
1863   2415 days
Name: date, Length: 1864, dtype: timedelta64[ns]

ufc_unified_rules.rename(columns = {'date': 'date1'}, inplace = True)

In [23]:
ufc_unified_rules.rename(columns = {'date': 'fight date'}, inplace = True)

/opt/anaconda3/envs/geospatial/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [24]:
ufc_unified_rules = pd.concat([ufc_unified_rules, r_time_active.reindex(ufc_unified_rules.index)], axis = 1)

In [25]:
ufc_unified_rules.rename(columns = {'date': 'r time active'}, inplace = True)

In [26]:
ufc_unified_rules = pd.concat([ufc_unified_rules, b_time_active.reindex(ufc_unified_rules.index)], axis = 1)

In [27]:
ufc_unified_rules.rename(columns = {'date': 'b time active'}, inplace = True)

In [28]:
ufc_unified_rules

,R_fighter,B_fighter,Referee,fight date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_index_ape,B_index_ape,R_index_ape2,B_index_ape2,r time active,b time active
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.0,...,177.80,135.0,31.0,27.0,1.04,1.03,7.62,5.08,0 days,0 days
1,Trevin Giles,Roman Dolidze,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.0,...,187.96,185.0,32.0,28.0,1.03,1.03,5.08,5.08,0 days,0 days
2,Tai Tuivasa,Harry Hunsucker,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,...,190.50,264.0,32.0,28.0,1.01,1.01,2.54,2.54,1715 days,2244 days
3,Cheyanne Buys,Montserrat Conejo,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,...,160.02,115.0,28.0,25.0,1.0,1.02,0.00,2.54,0 days,3424 days
4,Marion Reneau,Macy Chiasson,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.0,...,172.72,135.0,29.0,43.0,1.03,1.01,5.08,2.54,1283 days,265 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5804,Scott Adams,Ian Freeman,Dan Severn,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Heavyweight,NaN,NaN,...,NaN,225.0,33.0,NaN,0,0,-182.88,-180.34,NaT,NaT
5805,Jens Pulver,David Velasquez,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Lightweight,NaN,NaN,...,177.80,145.0,NaN,25.0,1.04,0,7.62,-165.10,NaT,NaT
5806,Tedd Williams,Steve Judson,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Heavyweight,0.000,1.0,...,NaN,275.0,NaN,30.0,0,0,-185.42,-180.34,NaT,NaT
5807,Bob Cook,Tiki Ghosn,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Lightweight,NaN,NaN,...,NaN,170.0,23.0,NaN,0,0,-175.26,-177.80,NaT,NaT


In [29]:
ufc_unified_rules.to_csv ('ufc_unified_rules.csv', index = False)

In [30]:
title_data = ufc_unified_rules[['R_fighter', 'B_fighter', 'title_bout', 'fight date', 'weight_class', 'Winner']]

In [31]:
title_data

,R_fighter,B_fighter,title_bout,fight date,weight_class,Winner
0,Adrian Yanez,Gustavo Lopez,False,2021-03-20,Bantamweight,Red
1,Trevin Giles,Roman Dolidze,False,2021-03-20,Middleweight,Red
2,Tai Tuivasa,Harry Hunsucker,False,2021-03-20,Heavyweight,Red
3,Cheyanne Buys,Montserrat Conejo,False,2021-03-20,WomenStrawweight,Blue
4,Marion Reneau,Macy Chiasson,False,2021-03-20,WomenBantamweight,Blue
...,...,...,...,...,...,...
5804,Scott Adams,Ian Freeman,False,2000-03-10,Heavyweight,Red
5805,Jens Pulver,David Velasquez,False,2000-03-10,Lightweight,Red
5806,Tedd Williams,Steve Judson,False,2000-03-10,Heavyweight,Red
5807,Bob Cook,Tiki Ghosn,False,2000-03-10,Lightweight,Red


In [32]:
title_data.to_csv ('title_data.csv', index = False)

In [33]:
r_fighter_stats = ufc_unified_rules.drop(columns = ['B_fighter', 'B_avg_KD', 'B_avg_opp_KD', 'B_avg_SIG_STR_pct', 
                                         'B_avg_opp_SIG_STR_pct', 'B_avg_TD_pct', 'B_avg_opp_TD_pct', 
                                         'B_avg_SUB_ATT', 'B_avg_opp_SUB_ATT', 'B_avg_REV', 'B_avg_opp_REV', 
                                         'B_avg_SIG_STR_att', 'B_avg_SIG_STR_landed', 'B_avg_opp_SIG_STR_att', 
                                         'B_avg_opp_SIG_STR_landed', 'B_avg_TOTAL_STR_att', 'B_avg_TOTAL_STR_landed', 
                                         'B_avg_opp_TOTAL_STR_att', 'B_avg_opp_TOTAL_STR_landed', 'B_avg_TD_att', 
                                         'B_avg_TD_landed', 'B_avg_opp_TD_att', 'B_avg_opp_TD_landed', 'B_avg_HEAD_att', 
                                         'B_avg_HEAD_landed', 'B_avg_opp_HEAD_att', 'B_avg_opp_HEAD_landed', 
                                         'B_avg_BODY_att', 'B_avg_BODY_landed', 'B_avg_opp_BODY_att', 
                                         'B_avg_opp_BODY_landed', 'B_avg_LEG_att', 'B_avg_LEG_landed', 
                                         'B_avg_opp_LEG_att', 'B_avg_opp_LEG_landed', 'B_avg_DISTANCE_att', 
                                         'B_avg_DISTANCE_landed', 'B_avg_opp_DISTANCE_att', 'B_avg_opp_DISTANCE_landed', 
                                         'B_avg_CLINCH_att', 'B_avg_CLINCH_landed', 'B_avg_opp_CLINCH_att', 
                                         'B_avg_opp_CLINCH_landed', 'B_avg_GROUND_att', 'B_avg_GROUND_landed', 
                                         'B_avg_opp_GROUND_att', 'B_avg_opp_GROUND_landed', 'B_avg_CTRL_time(seconds)',
                                         'B_avg_opp_CTRL_time(seconds)', 'B_total_time_fought(seconds)', 
                                         'B_total_rounds_fought', 'B_total_title_bouts', 'B_current_win_streak', 
                                         'B_current_lose_streak', 'B_longest_win_streak', 'B_wins', 'B_losses', 
                                         'B_draw', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 
                                         'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 
                                         'B_win_by_TKO_Doctor_Stoppage', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 
                                         'B_Weight_lbs', 'B_age', 'B_index_ape', 'B_index_ape2', 'b time active'])

In [34]:
r_fighter_stats

,R_fighter,Referee,fight date,location,Winner,title_bout,weight_class,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,...,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,R_index_ape,R_index_ape2,r time active
0,Adrian Yanez,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,1.000000,0.000000,0.500000,...,0,0,Orthodox,170.18,177.80,135.0,27.0,1.04,7.62,0 days
1,Trevin Giles,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,1.031250,0.062500,0.576875,...,0,0,Orthodox,182.88,187.96,185.0,28.0,1.03,5.08,0 days
2,Tai Tuivasa,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,0.546875,0.187500,0.538906,...,0,0,Southpaw,187.96,190.50,264.0,28.0,1.01,2.54,1715 days
3,Cheyanne Buys,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,NaN,...,0,0,Switch,160.02,160.02,115.0,25.0,1.0,0.00,0 days
4,Marion Reneau,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.000000,0.000977,0.403076,...,2,0,Orthodox,167.64,172.72,135.0,43.0,1.03,5.08,1283 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5804,Scott Adams,Dan Severn,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Heavyweight,NaN,NaN,NaN,...,0,0,Southpaw,182.88,NaN,225.0,NaN,0,-182.88,NaT
5805,Jens Pulver,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Lightweight,0.000000,0.000000,0.590000,...,0,0,Southpaw,170.18,177.80,145.0,25.0,1.04,7.62,NaT
5806,Tedd Williams,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Heavyweight,NaN,NaN,NaN,...,0,0,Orthodox,185.42,NaN,275.0,30.0,0,-185.42,NaT
5807,Bob Cook,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Lightweight,NaN,NaN,NaN,...,0,0,Orthodox,175.26,NaN,170.0,NaN,0,-175.26,NaT


In [35]:
test = r_fighter_stats[['R_wins', 'R_draw', 'R_losses']]

In [36]:
test

,R_wins,R_draw,R_losses
0,1,0,0
1,4,0,2
2,4,0,3
3,0,0,0
4,5,0,6
...,...,...,...
5804,0,0,0
5805,0,0,1
5806,0,0,0
5807,0,0,0


In [37]:
for col in r_fighter_stats.columns:
    print(col)

R_fighter
Referee
fight date
location
Winner
title_bout
weight_class
R_avg_KD
R_avg_opp_KD
R_avg_SIG_STR_pct
R_avg_opp_SIG_STR_pct
R_avg_TD_pct
R_avg_opp_TD_pct
R_avg_SUB_ATT
R_avg_opp_SUB_ATT
R_avg_REV
R_avg_opp_REV
R_avg_SIG_STR_att
R_avg_SIG_STR_landed
R_avg_opp_SIG_STR_att
R_avg_opp_SIG_STR_landed
R_avg_TOTAL_STR_att
R_avg_TOTAL_STR_landed
R_avg_opp_TOTAL_STR_att
R_avg_opp_TOTAL_STR_landed
R_avg_TD_att
R_avg_TD_landed
R_avg_opp_TD_att
R_avg_opp_TD_landed
R_avg_HEAD_att
R_avg_HEAD_landed
R_avg_opp_HEAD_att
R_avg_opp_HEAD_landed
R_avg_BODY_att
R_avg_BODY_landed
R_avg_opp_BODY_att
R_avg_opp_BODY_landed
R_avg_LEG_att
R_avg_LEG_landed
R_avg_opp_LEG_att
R_avg_opp_LEG_landed
R_avg_DISTANCE_att
R_avg_DISTANCE_landed
R_avg_opp_DISTANCE_att
R_avg_opp_DISTANCE_landed
R_avg_CLINCH_att
R_avg_CLINCH_landed
R_avg_opp_CLINCH_att
R_avg_opp_CLINCH_landed
R_avg_GROUND_att
R_avg_GROUND_landed
R_avg_opp_GROUND_att
R_avg_opp_GROUND_landed
R_avg_CTRL_time(seconds)
R_avg_opp_CTRL_time(seconds)
R_total_tim

In [38]:
r_fighter_stats[['R_fighter', 'R_wins', 'R_draw', 'R_losses']]

,R_fighter,R_wins,R_draw,R_losses
0,Adrian Yanez,1,0,0
1,Trevin Giles,4,0,2
2,Tai Tuivasa,4,0,3
3,Cheyanne Buys,0,0,0
4,Marion Reneau,5,0,6
...,...,...,...,...
5804,Scott Adams,0,0,0
5805,Jens Pulver,0,0,1
5806,Tedd Williams,0,0,0
5807,Bob Cook,0,0,0


In [39]:
r_fighter_stats[r_fighter_stats['R_fighter'] == 'Dustin Poirier']

,R_fighter,Referee,fight date,location,Winner,title_bout,weight_class,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,...,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,R_index_ape,R_index_ape2,r time active
85,Dustin Poirier,Herb Dean,2021-01-23,"Abu Dhabi, Abu Dhabi, United Arab Emirates",Red,False,Lightweight,0.065247,0.002014,0.522531,...,3,0,Southpaw,175.26,182.88,155.0,32.0,1.04,7.62,1345 days
395,Dustin Poirier,Herb Dean,2020-06-27,"Las Vegas, Nevada, USA",Red,False,Lightweight,0.130493,0.004028,0.385061,...,3,0,Southpaw,175.26,182.88,155.0,31.0,1.04,7.62,607 days
1437,Dustin Poirier,Herb Dean,2018-04-14,"Glendale, Arizona, USA",Red,False,Lightweight,0.087891,0.064453,0.510980,...,3,0,Southpaw,175.26,182.88,155.0,29.0,1.04,7.62,0 days
1650,Dustin Poirier,Keith Peterson,2017-11-11,"Norfolk, Virginia, USA",Red,False,Lightweight,0.175781,0.128906,0.491959,...,3,0,Southpaw,175.26,182.88,155.0,28.0,1.04,7.62,NaT
1976,Dustin Poirier,Todd Ronald Anderson,2017-02-11,"Brooklyn, New York, USA",Red,False,Lightweight,0.703125,0.515625,0.527837,...,3,0,Southpaw,175.26,182.88,155.0,28.0,1.04,7.62,NaT
2176,Dustin Poirier,Dan Miragliotta,2016-09-17,"Hidalgo, Texas, USA",Blue,False,Lightweight,1.406250,0.031250,0.555674,...,3,0,Southpaw,175.26,182.88,155.0,27.0,1.04,7.62,NaT
2333,Dustin Poirier,Jason Herzog,2016-06-04,"Los Angeles, California, USA",Red,False,Lightweight,0.812500,0.062500,0.571348,...,3,0,Southpaw,175.26,182.88,155.0,27.0,1.04,7.62,NaT
2509,Dustin Poirier,Chris Tognoni,2016-01-02,"Las Vegas, Nevada, USA",Red,False,Lightweight,1.625000,0.125000,0.622695,...,3,0,Southpaw,175.26,182.88,155.0,26.0,1.04,7.62,NaT
2783,Dustin Poirier,John McCarthy,2015-06-06,"New Orleans, Louisiana, USA",Red,False,Lightweight,1.250000,0.250000,0.495391,...,3,0,Southpaw,175.26,182.88,155.0,26.0,1.04,7.62,NaT
2879,Dustin Poirier,Todd McGovern,2015-04-04,"Fairfax, Virginia, USA",Red,False,Lightweight,0.500000,0.500000,0.510781,...,3,0,Southpaw,175.26,182.88,155.0,26.0,1.04,7.62,NaT


b_fighter_stats[b_fighter_stats['B_fighter'] == 'Dustin Poirier']

In [40]:
r_fighter_stats.to_csv ('r_fighter_stats.csv', index = False)

In [41]:
b_fighter_stats = ufc_unified_rules.drop(columns = ['R_fighter', 'R_avg_KD', 'R_avg_opp_KD', 'R_avg_SIG_STR_pct', 
                                         'R_avg_opp_SIG_STR_pct', 'R_avg_TD_pct', 'R_avg_opp_TD_pct', 
                                         'R_avg_SUB_ATT', 'R_avg_opp_SUB_ATT', 'R_avg_REV', 'R_avg_opp_REV', 
                                         'R_avg_SIG_STR_att', 'R_avg_SIG_STR_landed', 'R_avg_opp_SIG_STR_att', 
                                         'R_avg_opp_SIG_STR_landed', 'R_avg_TOTAL_STR_att', 'R_avg_TOTAL_STR_landed', 
                                         'R_avg_opp_TOTAL_STR_att', 'R_avg_opp_TOTAL_STR_landed', 'R_avg_TD_att', 
                                         'R_avg_TD_landed', 'R_avg_opp_TD_att', 'R_avg_opp_TD_landed', 'R_avg_HEAD_att', 
                                         'R_avg_HEAD_landed', 'R_avg_opp_HEAD_att', 'R_avg_opp_HEAD_landed', 
                                         'R_avg_BODY_att', 'R_avg_BODY_landed', 'R_avg_opp_BODY_att', 
                                         'R_avg_opp_BODY_landed', 'R_avg_LEG_att', 'R_avg_LEG_landed', 
                                         'R_avg_opp_LEG_att', 'R_avg_opp_LEG_landed', 'R_avg_DISTANCE_att', 
                                         'R_avg_DISTANCE_landed', 'R_avg_opp_DISTANCE_att', 'R_avg_opp_DISTANCE_landed', 
                                         'R_avg_CLINCH_att', 'R_avg_CLINCH_landed', 'R_avg_opp_CLINCH_att', 
                                         'R_avg_opp_CLINCH_landed', 'R_avg_GROUND_att', 'R_avg_GROUND_landed', 
                                         'R_avg_opp_GROUND_att', 'R_avg_opp_GROUND_landed', 'R_avg_CTRL_time(seconds)',
                                         'R_avg_opp_CTRL_time(seconds)', 'R_total_time_fought(seconds)', 
                                         'R_total_rounds_fought', 'R_total_title_bouts', 'R_current_win_streak', 
                                         'R_current_lose_streak', 'R_longest_win_streak', 'R_wins', 'R_losses', 
                                         'R_draw', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 
                                         'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission', 
                                         'R_win_by_TKO_Doctor_Stoppage', 'R_Stance', 'R_Height_cms', 'R_Reach_cms', 
                                         'R_Weight_lbs', 'R_age', 'R_index_ape', 'R_index_ape2', 'r time active'])

In [42]:
b_fighter_stats

,B_fighter,Referee,fight date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,B_age,B_index_ape,B_index_ape2,b time active
0,Gustavo Lopez,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.0,0.420000,...,1,0,Orthodox,165.10,170.18,135.0,31.0,1.03,5.08,0 days
1,Roman Dolidze,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.0,0.660000,...,0,0,Orthodox,187.96,193.04,205.0,32.0,1.03,5.08,0 days
2,Harry Hunsucker,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,NaN,...,0,0,Orthodox,187.96,190.50,241.0,32.0,1.01,2.54,2244 days
3,Montserrat Conejo,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,NaN,...,0,0,Southpaw,152.40,154.94,115.0,28.0,1.02,2.54,3424 days
4,Macy Chiasson,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.0,0.535625,...,1,0,Orthodox,180.34,182.88,135.0,29.0,1.01,2.54,265 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5804,Ian Freeman,Dan Severn,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Heavyweight,NaN,NaN,NaN,...,0,0,Orthodox,180.34,NaN,225.0,33.0,0,-180.34,NaT
5805,David Velasquez,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Lightweight,NaN,NaN,NaN,...,0,0,Orthodox,165.10,NaN,145.0,NaN,0,-165.10,NaT
5806,Steve Judson,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Heavyweight,0.000,1.0,0.000000,...,0,0,Orthodox,180.34,NaN,230.0,NaN,0,-180.34,NaT
5807,Tiki Ghosn,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Lightweight,NaN,NaN,NaN,...,0,0,Orthodox,177.80,NaN,170.0,23.0,0,-177.80,NaT


In [43]:
test2 = b_fighter_stats[['B_wins', 'B_draw', 'B_losses']]

In [44]:
test2

,B_wins,B_draw,B_losses
0,1,0,1
1,2,0,0
2,0,0,0
3,0,0,0
4,4,0,1
...,...,...,...
5804,0,0,0
5805,0,0,0
5806,0,0,1
5807,0,0,0


In [45]:
active_fighters = pd.read_csv('fighter_df.csv')

In [46]:
for col in active_fighters.columns:
    print(col)

FighterId
FirstName
LastName
Nickname
WeightClass
BirthDate
Height
Weight
Reach
Wins
Losses
Draws
NoContests
TechnicalKnockouts
TechnicalKnockoutLosses
Submissions
SubmissionLosses
TitleWins
TitleLosses
TitleDraws
CareerStats.FighterId
CareerStats.FirstName
CareerStats.LastName
CareerStats.SigStrikesLandedPerMinute
CareerStats.SigStrikeAccuracy
CareerStats.TakedownAverage
CareerStats.SubmissionAverage
CareerStats.KnockoutPercentage
CareerStats.TechnicalKnockoutPercentage
CareerStats.DecisionPercentage
CareerStats


In [47]:
active_fighters

,FighterId,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,Reach,Wins,...,CareerStats.FirstName,CareerStats.LastName,CareerStats.SigStrikesLandedPerMinute,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage,CareerStats
0,140000001,Chance,Rencountre,Black Eagle,Welterweight,1986-12-31T00:00:00,74.0,170.0,75.0,14.0,...,Chance,Rencountre,2.6,31.7,3.2,0.8,0.0,0.0,13.3,NaN
1,140000002,Kyle,Stewart,Gunz Up,Welterweight,1989-01-31T00:00:00,72.0,170.0,76.5,13.0,...,Kyle,Stewart,3.4,54.3,3.1,0.0,0.0,0.0,0.0,NaN
2,140000003,Geoff,Neal,Handz of Steel,Welterweight,1990-08-28T00:00:00,71.0,171.0,75.0,13.0,...,Geoff,Neal,10.5,80.4,1.4,0.5,36.3,36.3,14.0,NaN
3,140000004,Belal,Muhammad,Remember the Name,Welterweight,1988-07-09T00:00:00,71.0,170.0,72.0,19.0,...,Belal,Muhammad,11.0,77.4,4.3,0.4,10.4,10.4,53.7,NaN
4,140000005,Dennis,Bermudez,The Menace,Featherweight,1986-12-13T00:00:00,66.0,155.0,66.0,17.0,...,Dennis,Bermudez,10.6,70.8,9.1,2.4,8.8,8.8,40.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,140001170,Bruno,Korea,NaN,Flyweight,1991-06-28T00:00:00,66.0,125.0,65.5,12.0,...,Bruno,Korea,4.8,73.4,0.0,0.0,0.0,0.0,0.0,NaN
1159,140001171,C.J.,Vergara,C.J.,Bantamweight,1991-06-18T00:00:00,66.0,135.0,69.5,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1160,140001172,Javid,Basharat,The Snow Leopard,Bantamweight,NaN,67.0,135.0,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1161,140001173,Saimon,Oliveira,NaN,Bantamweight,1991-04-21T00:00:00,69.0,135.0,67.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
total_fighter_weightclass = active_fighters[["WeightClass"]]

In [49]:
total_fighter_weightclass.value_counts()

WeightClass          
Lightweight              155
Welterweight             151
Featherweight            146
Bantamweight             134
Middleweight             111
Light Heavyweight         89
Heavyweight               87
Women's Strawweight       67
Women's Flyweight         57
Flyweight                 55
Women's Bantamweight      41
Women's Featherweight     10
Catch Weight               1
Catchweight                1
dtype: int64

In [50]:
total_fighter_weightclass.to_csv ('total_fighter_weightclass.csv', index = False)

In [51]:
r_total_fights = r_fighter_stats["R_wins"] + r_fighter_stats["R_losses"] + r_fighter_stats["R_draw"]

In [52]:
r_total_fights

0        1
1        6
2        7
3        0
4       11
        ..
5804     0
5805     1
5806     0
5807     0
5808     0
Length: 5809, dtype: int64

In [53]:
r_total_fights.to_csv ('r_total_fights.csv', index = False)

In [54]:
r_fighter_stats["total_fights"] = r_total_fights

In [55]:
r_win_pct = ((r_fighter_stats["R_wins"] / r_fighter_stats["total_fights"]) * 100)

In [56]:
r_fighter_stats["win_pct"] = r_win_pct

In [57]:
r_win_pct

0       100.000000
1        66.666667
2        57.142857
3              NaN
4        45.454545
           ...    
5804           NaN
5805      0.000000
5806           NaN
5807           NaN
5808           NaN
Length: 5809, dtype: float64

In [58]:
r_fighter_stats

,R_fighter,Referee,fight date,location,Winner,title_bout,weight_class,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,...,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,R_index_ape,R_index_ape2,r time active,total_fights,win_pct
0,Adrian Yanez,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,1.000000,0.000000,0.500000,...,Orthodox,170.18,177.80,135.0,27.0,1.04,7.62,0 days,1,100.000000
1,Trevin Giles,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,1.031250,0.062500,0.576875,...,Orthodox,182.88,187.96,185.0,28.0,1.03,5.08,0 days,6,66.666667
2,Tai Tuivasa,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,0.546875,0.187500,0.538906,...,Southpaw,187.96,190.50,264.0,28.0,1.01,2.54,1715 days,7,57.142857
3,Cheyanne Buys,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,NaN,...,Switch,160.02,160.02,115.0,25.0,1.0,0.00,0 days,0,NaN
4,Marion Reneau,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.000000,0.000977,0.403076,...,Orthodox,167.64,172.72,135.0,43.0,1.03,5.08,1283 days,11,45.454545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5804,Scott Adams,Dan Severn,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Heavyweight,NaN,NaN,NaN,...,Southpaw,182.88,NaN,225.0,NaN,0,-182.88,NaT,0,NaN
5805,Jens Pulver,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Lightweight,0.000000,0.000000,0.590000,...,Southpaw,170.18,177.80,145.0,25.0,1.04,7.62,NaT,1,0.000000
5806,Tedd Williams,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Heavyweight,NaN,NaN,NaN,...,Orthodox,185.42,NaN,275.0,30.0,0,-185.42,NaT,0,NaN
5807,Bob Cook,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Lightweight,NaN,NaN,NaN,...,Orthodox,175.26,NaN,170.0,NaN,0,-175.26,NaT,0,NaN


In [59]:
r_fighter_stats.to_csv ('r_fighter_stats.csv', index = False)

In [60]:
b_total_fights = b_fighter_stats["B_wins"] + b_fighter_stats["B_losses"] + b_fighter_stats["B_draw"]

In [61]:
b_total_fights

0       2
1       2
2       0
3       0
4       5
       ..
5804    0
5805    0
5806    1
5807    0
5808    0
Length: 5809, dtype: int64

In [62]:
b_total_fights.to_csv ('b_total_fights.csv', index = False)

In [63]:
b_fighter_stats["total_fights"] = b_total_fights

In [64]:
b_win_pct = ((b_fighter_stats["B_wins"] / b_fighter_stats["total_fights"]) * 100)

In [65]:
b_fighter_stats["win_pct"] = b_win_pct

In [66]:
b_win_pct

0        50.0
1       100.0
2         NaN
3         NaN
4        80.0
        ...  
5804      NaN
5805      NaN
5806      0.0
5807      NaN
5808      NaN
Length: 5809, dtype: float64

In [67]:
b_fighter_stats

,B_fighter,Referee,fight date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,B_age,B_index_ape,B_index_ape2,b time active,total_fights,win_pct
0,Gustavo Lopez,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.0,0.420000,...,Orthodox,165.10,170.18,135.0,31.0,1.03,5.08,0 days,2,50.0
1,Roman Dolidze,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.0,0.660000,...,Orthodox,187.96,193.04,205.0,32.0,1.03,5.08,0 days,2,100.0
2,Harry Hunsucker,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,NaN,...,Orthodox,187.96,190.50,241.0,32.0,1.01,2.54,2244 days,0,NaN
3,Montserrat Conejo,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,NaN,...,Southpaw,152.40,154.94,115.0,28.0,1.02,2.54,3424 days,0,NaN
4,Macy Chiasson,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.0,0.535625,...,Orthodox,180.34,182.88,135.0,29.0,1.01,2.54,265 days,5,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5804,Ian Freeman,Dan Severn,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Heavyweight,NaN,NaN,NaN,...,Orthodox,180.34,NaN,225.0,33.0,0,-180.34,NaT,0,NaN
5805,David Velasquez,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Lightweight,NaN,NaN,NaN,...,Orthodox,165.10,NaN,145.0,NaN,0,-165.10,NaT,0,NaN
5806,Steve Judson,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Heavyweight,0.000,1.0,0.000000,...,Orthodox,180.34,NaN,230.0,NaN,0,-180.34,NaT,1,0.0
5807,Tiki Ghosn,John McCarthy,2000-03-10,"Lake Charles, Louisiana, USA",Red,False,Lightweight,NaN,NaN,NaN,...,Orthodox,177.80,NaN,170.0,23.0,0,-177.80,NaT,0,NaN


In [68]:
b_fighter_stats.to_csv ('b_fighter_stats.csv', index = False)

In [69]:
active_fighters = active_fighters.drop(columns = ['CareerStats.FighterId',
                                                  'CareerStats.FirstName', 
                                                  'CareerStats.LastName'])

In [70]:
active_fighters['BirthDate'] = pd.to_datetime(active_fighters['BirthDate'])

In [71]:
active_fighters

,FighterId,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,Reach,Wins,...,TitleLosses,TitleDraws,CareerStats.SigStrikesLandedPerMinute,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage,CareerStats
0,140000001,Chance,Rencountre,Black Eagle,Welterweight,1986-12-31,74.0,170.0,75.0,14.0,...,0.0,0.0,2.6,31.7,3.2,0.8,0.0,0.0,13.3,NaN
1,140000002,Kyle,Stewart,Gunz Up,Welterweight,1989-01-31,72.0,170.0,76.5,13.0,...,2.0,0.0,3.4,54.3,3.1,0.0,0.0,0.0,0.0,NaN
2,140000003,Geoff,Neal,Handz of Steel,Welterweight,1990-08-28,71.0,171.0,75.0,13.0,...,0.0,0.0,10.5,80.4,1.4,0.5,36.3,36.3,14.0,NaN
3,140000004,Belal,Muhammad,Remember the Name,Welterweight,1988-07-09,71.0,170.0,72.0,19.0,...,0.0,0.0,11.0,77.4,4.3,0.4,10.4,10.4,53.7,NaN
4,140000005,Dennis,Bermudez,The Menace,Featherweight,1986-12-13,66.0,155.0,66.0,17.0,...,0.0,0.0,10.6,70.8,9.1,2.4,8.8,8.8,40.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,140001170,Bruno,Korea,NaN,Flyweight,1991-06-28,66.0,125.0,65.5,12.0,...,0.0,0.0,4.8,73.4,0.0,0.0,0.0,0.0,0.0,NaN
1159,140001171,C.J.,Vergara,C.J.,Bantamweight,1991-06-18,66.0,135.0,69.5,8.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1160,140001172,Javid,Basharat,The Snow Leopard,Bantamweight,NaT,67.0,135.0,0.0,10.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1161,140001173,Saimon,Oliveira,NaN,Bantamweight,1991-04-21,69.0,135.0,67.0,17.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
print(active_fighters[['Nickname']])

               Nickname
0           Black Eagle
1               Gunz Up
2        Handz of Steel
3     Remember the Name
4            The Menace
...                 ...
1158                NaN
1159               C.J.
1160   The Snow Leopard
1161                NaN
1162    Chidi Bang Bang

[1163 rows x 1 columns]


In [73]:
nickname_list = active_fighters.index.tolist()

In [74]:
fighter_nickname = active_fighters["Nickname"].tolist()

In [75]:
total_fighter_nickname_count = Counter(fighter_nickname)

In [76]:
total_fighter_nickname_count = total_fighter_nickname_count.most_common()

In [77]:
total_fighter_nickname_count

[(nan, 347),
 ('The Spartan', 7),
 ('The Great', 4),
 ('Pitbull', 4),
 ('Danger', 4),
 ('The Sniper', 4),
 ('The Predator', 3),
 ('El Toro', 3),
 ('Bullet', 3),
 ('Wolverine', 3),
 ('The Beast', 3),
 ('The Menace', 2),
 ('Cowboy', 2),
 ('The Kid', 2),
 ('Pretty Boy', 2),
 ('Cyborg', 2),
 ('Junior', 2),
 ('Magic', 2),
 ('Dragon', 2),
 ('The Tarantula', 2),
 ('Golden Boy', 2),
 ('Hot Sauce', 2),
 ('Fury', 2),
 ('Bam Bam', 2),
 ('The Savage', 2),
 ('Kid Dynamite', 2),
 ('The Body Snatcher', 2),
 ('The Barbarian', 2),
 ('Lights Out', 2),
 ('Rocky', 2),
 ('Showtime', 2),
 ('El Guapo', 2),
 ('The Future', 2),
 ('Razor', 2),
 ('Super', 2),
 ('The Warrior', 2),
 ('La Pantera', 2),
 ('The Demolition Man', 2),
 ('The Monster', 2),
 ('The Machine', 2),
 ('Superman', 2),
 ('Hurricane', 2),
 ('The Prodigy', 2),
 ('The Lone Wolf', 2),
 ('The Hurricane', 2),
 ('Triple Threat', 2),
 ('The Bully', 2),
 ('Mr. Highlight', 2),
 ('Flash', 2),
 ('El Nino', 2),
 ('All In', 2),
 ('Lucky', 2),
 ('Grizzly', 2),

In [78]:
total_nickname_count = pd.DataFrame(total_fighter_nickname_count, columns = ['nickname', 'count'])
print(total_fighter_nickname_count)

[(nan, 347), ('The Spartan', 7), ('The Great', 4), ('Pitbull', 4), ('Danger', 4), ('The Sniper', 4), ('The Predator', 3), ('El Toro', 3), ('Bullet', 3), ('Wolverine', 3), ('The Beast', 3), ('The Menace', 2), ('Cowboy', 2), ('The Kid', 2), ('Pretty Boy', 2), ('Cyborg', 2), ('Junior', 2), ('Magic', 2), ('Dragon', 2), ('The Tarantula', 2), ('Golden Boy', 2), ('Hot Sauce', 2), ('Fury', 2), ('Bam Bam', 2), ('The Savage', 2), ('Kid Dynamite', 2), ('The Body Snatcher', 2), ('The Barbarian', 2), ('Lights Out', 2), ('Rocky', 2), ('Showtime', 2), ('El Guapo', 2), ('The Future', 2), ('Razor', 2), ('Super', 2), ('The Warrior', 2), ('La Pantera', 2), ('The Demolition Man', 2), ('The Monster', 2), ('The Machine', 2), ('Superman', 2), ('Hurricane', 2), ('The Prodigy', 2), ('The Lone Wolf', 2), ('The Hurricane', 2), ('Triple Threat', 2), ('The Bully', 2), ('Mr. Highlight', 2), ('Flash', 2), ('El Nino', 2), ('All In', 2), ('Lucky', 2), ('Grizzly', 2), ('Soldier of God', 2), ('Thunder', 2), ('The Leech'

In [79]:
total_nickname_count.head(10)

,nickname,count
0,NaN,347
1,The Spartan,7
2,The Great,4
3,Pitbull,4
4,Danger,4
5,The Sniper,4
6,The Predator,3
7,El Toro,3
8,Bullet,3
9,Wolverine,3


In [80]:
total_nickname_count['count'].dtype

dtype('int64')

In [81]:
total_nickname_count['nickname'].dtype

dtype('O')

In [82]:
total_nickname_count.to_csv ('total_nickname_count.csv', index = False)

In [83]:
now = pd.Timestamp('now')
active_fighters['BirthDate'] = pd.to_datetime(active_fighters['BirthDate'], format = '%m%d%y')
active_fighters['BirthDate'] = active_fighters['BirthDate'].where(active_fighters['BirthDate'] < now, active_fighters['BirthDate'] -  np.timedelta64(100, 'Y'))
active_fighters['age'] = (now - active_fighters['BirthDate']).astype('<m8[Y]')

In [84]:
active_fighters

,FighterId,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,Reach,Wins,...,TitleDraws,CareerStats.SigStrikesLandedPerMinute,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage,CareerStats,age
0,140000001,Chance,Rencountre,Black Eagle,Welterweight,1986-12-31,74.0,170.0,75.0,14.0,...,0.0,2.6,31.7,3.2,0.8,0.0,0.0,13.3,NaN,34.0
1,140000002,Kyle,Stewart,Gunz Up,Welterweight,1989-01-31,72.0,170.0,76.5,13.0,...,0.0,3.4,54.3,3.1,0.0,0.0,0.0,0.0,NaN,32.0
2,140000003,Geoff,Neal,Handz of Steel,Welterweight,1990-08-28,71.0,171.0,75.0,13.0,...,0.0,10.5,80.4,1.4,0.5,36.3,36.3,14.0,NaN,31.0
3,140000004,Belal,Muhammad,Remember the Name,Welterweight,1988-07-09,71.0,170.0,72.0,19.0,...,0.0,11.0,77.4,4.3,0.4,10.4,10.4,53.7,NaN,33.0
4,140000005,Dennis,Bermudez,The Menace,Featherweight,1986-12-13,66.0,155.0,66.0,17.0,...,0.0,10.6,70.8,9.1,2.4,8.8,8.8,40.0,NaN,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,140001170,Bruno,Korea,NaN,Flyweight,1991-06-28,66.0,125.0,65.5,12.0,...,0.0,4.8,73.4,0.0,0.0,0.0,0.0,0.0,NaN,30.0
1159,140001171,C.J.,Vergara,C.J.,Bantamweight,1991-06-18,66.0,135.0,69.5,8.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0
1160,140001172,Javid,Basharat,The Snow Leopard,Bantamweight,NaT,67.0,135.0,0.0,10.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1161,140001173,Saimon,Oliveira,NaN,Bantamweight,1991-04-21,69.0,135.0,67.0,17.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0


In [85]:
active_fighters["age"].mean()

31.762757385854968

In [86]:
active_fighters.sort_values(by = 'age', ascending = False)

,FighterId,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,Reach,Wins,...,TitleDraws,CareerStats.SigStrikesLandedPerMinute,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage,CareerStats,age
687,140000690,Mark,Hunt,Super Samoan,Heavyweight,1974-03-23,70.0,265.0,72.0,13.0,...,0.0,6.7,80.1,1.2,0.5,53.2,53.2,16.0,NaN,47.0
703,140000706,Mike,Pyle,Quicksand,Welterweight,1975-09-18,73.0,170.0,74.0,27.0,...,0.0,6.5,70.9,4.3,3.7,18.4,18.4,14.7,NaN,46.0
72,140000073,Anderson,Silva,The Spider,Middleweight,1975-04-14,74.0,185.0,77.0,34.0,...,0.0,7.3,95.0,1.2,1.9,63.8,63.8,26.9,NaN,46.0
341,140000342,Rogerio,Nogueira,Minotouro,Light Heavyweight,1976-06-02,74.0,205.0,75.0,23.0,...,0.0,5.7,53.7,1.6,2.0,37.4,37.4,32.7,NaN,45.0
717,140000720,Fabricio,Werdum,Vai Cavalo,Heavyweight,1977-07-30,76.0,241.0,77.0,24.0,...,0.0,8.6,94.6,4.2,2.8,27.3,27.3,27.3,NaN,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,140001164,Islam,Mamedov,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1153,140001165,Brent,Primus,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1154,140001166,Usman,Nurmagomedov,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1155,140001167,Luis,Muro,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
active_fighters.sort_values(by = 'age', ascending = True)

,FighterId,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,Reach,Wins,...,TitleDraws,CareerStats.SigStrikesLandedPerMinute,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage,CareerStats,age
964,140000970,Cory,McKenna,The Hobbit,Women's Strawweight,1999-11-07,63.0,115.0,58.0,6.0,...,0.0,8.3,69.2,1.2,0.0,0.0,0.0,22.1,NaN,21.0
1101,140001113,Rong,Zhu,NaN,Lightweight,2000-03-07,69.0,156.0,71.0,18.0,...,0.0,6.7,66.4,7.2,0.0,0.0,0.0,0.0,NaN,21.0
817,140000820,Chase,Hooper,The Dream,Featherweight,1999-09-13,73.0,145.0,74.0,10.0,...,0.0,9.2,90.0,2.3,5.1,11.9,11.9,11.9,NaN,22.0
926,140000930,Kay,Hansen,NaN,Women's Strawweight,1999-08-14,62.0,115.0,63.0,7.0,...,0.0,8.5,91.7,5.2,3.9,0.0,0.0,0.0,NaN,22.0
944,140000950,Luis,Rodriguez,Lazy Boy,Bantamweight,1999-05-07,66.0,125.0,65.5,11.0,...,0.0,3.0,57.5,4.8,4.8,0.0,0.0,0.0,NaN,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,140001164,Islam,Mamedov,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1153,140001165,Brent,Primus,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1154,140001166,Usman,Nurmagomedov,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1155,140001167,Luis,Muro,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
active_fighters["Fighter Name"] = active_fighters["FirstName"] + ' ' + active_fighters["LastName"]

In [89]:
active_fighters = active_fighters[["FighterId","Fighter Name", "age", "FirstName", "LastName", "Nickname",
                     "WeightClass", "BirthDate", "Height", "Weight", "Reach", "Wins", "Losses", "Draws", 
                     "NoContests", "TechnicalKnockouts", "TechnicalKnockoutLosses", "Submissions", 
                     "SubmissionLosses", "TitleWins", "TitleLosses", "TitleDraws", 
                     "CareerStats.SigStrikesLandedPerMinute", "CareerStats.SigStrikeAccuracy", 
                     "CareerStats.TakedownAverage", "CareerStats.SubmissionAverage",
                     "CareerStats.KnockoutPercentage", "CareerStats.TechnicalKnockoutPercentage", 
                     "CareerStats.DecisionPercentage"]]

In [90]:
active_fighters

,FighterId,Fighter Name,age,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,...,TitleWins,TitleLosses,TitleDraws,CareerStats.SigStrikesLandedPerMinute,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage
0,140000001,Chance Rencountre,34.0,Chance,Rencountre,Black Eagle,Welterweight,1986-12-31,74.0,170.0,...,0.0,0.0,0.0,2.6,31.7,3.2,0.8,0.0,0.0,13.3
1,140000002,Kyle Stewart,32.0,Kyle,Stewart,Gunz Up,Welterweight,1989-01-31,72.0,170.0,...,0.0,2.0,0.0,3.4,54.3,3.1,0.0,0.0,0.0,0.0
2,140000003,Geoff Neal,31.0,Geoff,Neal,Handz of Steel,Welterweight,1990-08-28,71.0,171.0,...,0.0,0.0,0.0,10.5,80.4,1.4,0.5,36.3,36.3,14.0
3,140000004,Belal Muhammad,33.0,Belal,Muhammad,Remember the Name,Welterweight,1988-07-09,71.0,170.0,...,2.0,0.0,0.0,11.0,77.4,4.3,0.4,10.4,10.4,53.7
4,140000005,Dennis Bermudez,34.0,Dennis,Bermudez,The Menace,Featherweight,1986-12-13,66.0,155.0,...,0.0,0.0,0.0,10.6,70.8,9.1,2.4,8.8,8.8,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,140001170,Bruno Korea,30.0,Bruno,Korea,NaN,Flyweight,1991-06-28,66.0,125.0,...,0.0,0.0,0.0,4.8,73.4,0.0,0.0,0.0,0.0,0.0
1159,140001171,C.J. Vergara,30.0,C.J.,Vergara,C.J.,Bantamweight,1991-06-18,66.0,135.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1160,140001172,Javid Basharat,NaN,Javid,Basharat,The Snow Leopard,Bantamweight,NaT,67.0,135.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1161,140001173,Saimon Oliveira,30.0,Saimon,Oliveira,NaN,Bantamweight,1991-04-21,69.0,135.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
total_fights = active_fighters["Wins"] + active_fighters["Losses"] + active_fighters["Draws"]

In [92]:
total_fights

0       18.0
1       16.0
2       17.0
3       22.0
4       26.0
        ... 
1158    16.0
1159    11.0
1160    10.0
1161    20.0
1162    26.0
Length: 1163, dtype: float64

In [93]:
active_fighters["total_fights"] = total_fights

In [94]:
win_pct = ((active_fighters["Wins"] / active_fighters["total_fights"]) * 100)

In [95]:
active_fighters["win_pct"] = win_pct

In [96]:
active_fighters

,FighterId,Fighter Name,age,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,...,TitleDraws,CareerStats.SigStrikesLandedPerMinute,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage,total_fights,win_pct
0,140000001,Chance Rencountre,34.0,Chance,Rencountre,Black Eagle,Welterweight,1986-12-31,74.0,170.0,...,0.0,2.6,31.7,3.2,0.8,0.0,0.0,13.3,18.0,77.777778
1,140000002,Kyle Stewart,32.0,Kyle,Stewart,Gunz Up,Welterweight,1989-01-31,72.0,170.0,...,0.0,3.4,54.3,3.1,0.0,0.0,0.0,0.0,16.0,81.250000
2,140000003,Geoff Neal,31.0,Geoff,Neal,Handz of Steel,Welterweight,1990-08-28,71.0,171.0,...,0.0,10.5,80.4,1.4,0.5,36.3,36.3,14.0,17.0,76.470588
3,140000004,Belal Muhammad,33.0,Belal,Muhammad,Remember the Name,Welterweight,1988-07-09,71.0,170.0,...,0.0,11.0,77.4,4.3,0.4,10.4,10.4,53.7,22.0,86.363636
4,140000005,Dennis Bermudez,34.0,Dennis,Bermudez,The Menace,Featherweight,1986-12-13,66.0,155.0,...,0.0,10.6,70.8,9.1,2.4,8.8,8.8,40.0,26.0,65.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,140001170,Bruno Korea,30.0,Bruno,Korea,NaN,Flyweight,1991-06-28,66.0,125.0,...,0.0,4.8,73.4,0.0,0.0,0.0,0.0,0.0,16.0,75.000000
1159,140001171,C.J. Vergara,30.0,C.J.,Vergara,C.J.,Bantamweight,1991-06-18,66.0,135.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,72.727273
1160,140001172,Javid Basharat,NaN,Javid,Basharat,The Snow Leopard,Bantamweight,NaT,67.0,135.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,100.000000
1161,140001173,Saimon Oliveira,30.0,Saimon,Oliveira,NaN,Bantamweight,1991-04-21,69.0,135.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,85.000000


In [97]:
title_fights = active_fighters["TitleWins"] + active_fighters["TitleLosses"] + active_fighters["TitleDraws"]

In [98]:
title_fights

0       0.0
1       2.0
2       0.0
3       2.0
4       0.0
       ... 
1158    0.0
1159    0.0
1160    0.0
1161    0.0
1162    2.0
Length: 1163, dtype: float64

In [99]:
active_fighters["title_fights"] = title_fights

In [100]:
title_win_pct = ((active_fighters["TitleWins"] / active_fighters["title_fights"]) * 100)

In [101]:
active_fighters["title_win_pct"] = title_win_pct

In [102]:
active_fighters

,FighterId,Fighter Name,age,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,...,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage,total_fights,win_pct,title_fights,title_win_pct
0,140000001,Chance Rencountre,34.0,Chance,Rencountre,Black Eagle,Welterweight,1986-12-31,74.0,170.0,...,31.7,3.2,0.8,0.0,0.0,13.3,18.0,77.777778,0.0,NaN
1,140000002,Kyle Stewart,32.0,Kyle,Stewart,Gunz Up,Welterweight,1989-01-31,72.0,170.0,...,54.3,3.1,0.0,0.0,0.0,0.0,16.0,81.250000,2.0,0.0
2,140000003,Geoff Neal,31.0,Geoff,Neal,Handz of Steel,Welterweight,1990-08-28,71.0,171.0,...,80.4,1.4,0.5,36.3,36.3,14.0,17.0,76.470588,0.0,NaN
3,140000004,Belal Muhammad,33.0,Belal,Muhammad,Remember the Name,Welterweight,1988-07-09,71.0,170.0,...,77.4,4.3,0.4,10.4,10.4,53.7,22.0,86.363636,2.0,100.0
4,140000005,Dennis Bermudez,34.0,Dennis,Bermudez,The Menace,Featherweight,1986-12-13,66.0,155.0,...,70.8,9.1,2.4,8.8,8.8,40.0,26.0,65.384615,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,140001170,Bruno Korea,30.0,Bruno,Korea,NaN,Flyweight,1991-06-28,66.0,125.0,...,73.4,0.0,0.0,0.0,0.0,0.0,16.0,75.000000,0.0,NaN
1159,140001171,C.J. Vergara,30.0,C.J.,Vergara,C.J.,Bantamweight,1991-06-18,66.0,135.0,...,NaN,NaN,NaN,NaN,NaN,NaN,11.0,72.727273,0.0,NaN
1160,140001172,Javid Basharat,NaN,Javid,Basharat,The Snow Leopard,Bantamweight,NaT,67.0,135.0,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,100.000000,0.0,NaN
1161,140001173,Saimon Oliveira,30.0,Saimon,Oliveira,NaN,Bantamweight,1991-04-21,69.0,135.0,...,NaN,NaN,NaN,NaN,NaN,NaN,20.0,85.000000,0.0,NaN


In [103]:
active_fighters[['Fighter Name', 'TitleWins', 'TitleLosses', 'TitleDraws', 'total_fights', 'Wins', 'Losses', 'Draws']]

,Fighter Name,TitleWins,TitleLosses,TitleDraws,total_fights,Wins,Losses,Draws
0,Chance Rencountre,0.0,0.0,0.0,18.0,14.0,4.0,0.0
1,Kyle Stewart,0.0,2.0,0.0,16.0,13.0,3.0,0.0
2,Geoff Neal,0.0,0.0,0.0,17.0,13.0,4.0,0.0
3,Belal Muhammad,2.0,0.0,0.0,22.0,19.0,3.0,0.0
4,Dennis Bermudez,0.0,0.0,0.0,26.0,17.0,9.0,0.0
...,...,...,...,...,...,...,...,...
1158,Bruno Korea,0.0,0.0,0.0,16.0,12.0,3.0,1.0
1159,C.J. Vergara,0.0,0.0,0.0,11.0,8.0,2.0,1.0
1160,Javid Basharat,0.0,0.0,0.0,10.0,10.0,0.0,0.0
1161,Saimon Oliveira,0.0,0.0,0.0,20.0,17.0,3.0,0.0


In [104]:
for col in active_fighters.columns:
    print(col)

FighterId
Fighter Name
age
FirstName
LastName
Nickname
WeightClass
BirthDate
Height
Weight
Reach
Wins
Losses
Draws
NoContests
TechnicalKnockouts
TechnicalKnockoutLosses
Submissions
SubmissionLosses
TitleWins
TitleLosses
TitleDraws
CareerStats.SigStrikesLandedPerMinute
CareerStats.SigStrikeAccuracy
CareerStats.TakedownAverage
CareerStats.SubmissionAverage
CareerStats.KnockoutPercentage
CareerStats.TechnicalKnockoutPercentage
CareerStats.DecisionPercentage
total_fights
win_pct
title_fights
title_win_pct


In [105]:
active_fighters.to_csv ('active_fighters.csv', index = False)